In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Provide spotify Credential Id, Secret Id, User Id stuff
cid = ''
secret = ''
user_id = ""

# Provide youtube api key, channel Id, Client Secret File
yt_api_key = ""
channel_id = ""
client_secrets_file = ""

# Initializing and Connecting to Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
# Getting the whole playlist related Information
my_play_list = sp.user_playlists(user_id, limit = 5)

In [7]:
# Getting all the playlist Id and Name
playlist_id = []
playlistName = []
print("All your current playlist:")
for item in my_play_list['items']:
    playlist_id.append(item['id'])
    playlistName.append(item['name'])
    print(item['name'])
    
# Getting Each playlist related infornamtion   
playlistTrackInfo = []
for i in range(len(playlist_id)):
    playlistTrackInfo.append(sp.user_playlist_tracks(user_id,playlist_id[i]))

All your current playlist:
//rock
Bluesway road
indie


In [8]:
# Creating a dictionary where key = playlist Number , value = song+artist namee
tracks = dict()

for i in range(len(playlistTrackInfo)):
    for item in playlistTrackInfo[i]['items']:
        track_name = item['track']['name'] + ' by ' + item['track']['artists'][0]['name']
        tracks.setdefault(i, []).append(track_name)

In [9]:
# Getting User Input [Which Spotify Playlist you want to Convert into a Youtube Playlist?]

for i in range (len(playlistName)):
    print("press " + str(i) + " for " + str(playlistName[i]))

print("Select Spotify Playlist: ")
selected_playlist = input()
print("you've selected playlist: " + playlistName[int(selected_playlist)])

press 0 for //rock
press 1 for Bluesway road
press 2 for indie
Select Spotify Playlist: 
0
you've selected playlist: //rock


In [10]:
# Initializing and Connecting Youtube API
from apiclient.discovery import build
youtube = build('youtube','v3', developerKey = yt_api_key)

In [20]:
# Getting all the songs from spotify playlist
# search it on youtube and select the 1st URL
# Take the video Ids of those tracks and store all of these informations
urls = []
kinds = []
video_ids = []
name_tracks_added = []

for track in tracks[int(selected_playlist)]:
    name_tracks_added.append(track)
    print(track + " Done.")
    req = youtube.search().list(q = track, part = 'snippet', type = 'video')
    res = req.execute()
    vid_id = res['items'][0]['id']['videoId']
    kind = res['items'][0]['id']['kind']
    kinds.append(kind)
    video_ids.append(vid_id)
    urls.append("https://www.youtube.com/watch?v=" + vid_id)

Metalingus by Alter Bridge Done.
In Loving Memory by Alter Bridge Done.
Godspeed by Alter Bridge Done.
The Fear by TRUSTcompany Done.
Hover by TRUSTcompany Done.


In [12]:
# Printing all the video ids of the searched tracks
video_ids

['N-I8ALlYxdY', 'qcQ3iegpZTc', 'UND4qN0baL0', '11bK1jqIbuA', 'ic2mJT0q5BU']

In [14]:
# Requesting to connect youtube api for getting the Channel ID
request = youtube.playlists().list(
    part = "snippet",
    channelId = channel_id,
    maxResults = 50
)
res = request.execute()

In [15]:
# Getting all the Playlist Names and ID of YOUTUBE
playlist_names_yt = []
playlist_id_yt = []

for channel in res['items']:
    playlist_names_yt.append(channel['snippet']['title'])
    playlist_id_yt.append(channel['id'])
print(playlist_names_yt)
print(playlist_id_yt)

['//rock_spotify', 'yesterdaySpotify', 'openSpot']
['PLGDiI5oeF_UghJbZq34TqtjDv_PL9HC-U', 'PLGDiI5oeF_Uh3WDT1TReJZAG0MB--p-jn', 'PLGDiI5oeF_Ui3FsorVTcxVKwQ0MgjzOJr']


In [16]:
# Select on Which Youtube Playlist you want to save all the Spotify tracks
for i in range (len(playlist_names_yt)):
    print("press " + str(i) + " for " + playlist_names_yt[i])
print("Enter Youtube playlist")
selecte_yt_playlist = input()
print("You've selected "+ playlist_names_yt[int(selecte_yt_playlist)])


selected_yt_playlist = playlist_id_yt[int(selecte_yt_playlist)]

press 0 for //rock_spotify
press 1 for yesterdaySpotify
press 2 for openSpot
Enter Youtube playlist
0
You've selected //rock_spotify


In [17]:
# THIS FOR GOOGLE O'AUTH STUFF
import google_auth_oauthlib.flow
scops =[""]
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scops)
credentials = flow.run_console()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=178586316215-gjb147i0953b41npbl8mnbnblgm8laeo.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=VvhMhw3815fvH9iGiGF52lzfWglykT&prompt=consent&access_type=offline
Enter the authorization code: 4/2gH2tDraQvTh592KCNN-8354ng-e6vgYJeDt-1e7l3fDjFcDyBDV4YQ


In [18]:
# Technical stuff (google o'auth, youtube data api credentials)
import googleapiclient.discovery
youtube = googleapiclient.discovery.build(
        "youtube", "v3",credentials=credentials)

In [21]:
# add all the spotify songs in youtube list
print("Selected Playlist Name: " + playlist_names_yt[int(selecte_yt_playlist)])

for index,vid in enumerate(video_ids):
    request = youtube.playlistItems().insert(part="snippet",
        body={
          "snippet": {
            "playlistId": selected_yt_playlist,
            "position": 0,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": vid
            }
          }
        }
    )
    response = request.execute()
    print(name_tracks_added[index] + " Added..")

Selected Playlist Name: //rock_spotify
Metalingus by Alter Bridge Done..
In Loving Memory by Alter Bridge Done..
Godspeed by Alter Bridge Done..
The Fear by TRUSTcompany Done..
Hover by TRUSTcompany Done..
